In [1]:
import pandas as pd
import numpy as np
from pivottablejs import pivot_ui

# Preprocessing

In [2]:
# Read in spreadsheets
data = pd.read_excel('data\Mfg Performance Data 05-31-2020.xlsx', sheet_name=None)

In [3]:
# Assign each spreadsheet to a dataframe and select columns
cr = data['Cetec Receipts'].loc[:, ['Vendor', 'Ext. Cost', 'Received On']]
csf = data['Cetec Shipped Filtered'].loc[:, ['Vendor', 'Ext Resale', 'Invoice Date']]
rr = data['Rz Receipts'].loc[:, ['Vendor Name', 'Total Cost', 'Invoice Date']]
rsf = data['Rz Shipped Filtered'].loc[:, ['Vendor Name', 'Total Price', 'Invoice Date']]

In [4]:
# Rename columns so they can be concatenated
cr.columns = ['Vendor', 'Receipts', 'Date']
csf.columns = ['Vendor', 'Shipments', 'Date']
rr.columns = ['Vendor', 'Receipts', 'Date']
rsf.columns = ['Vendor', 'Shipments', 'Date']

# Concatenate tables
report = pd.concat([cr, csf, rr, rsf], axis=0, ignore_index=True, sort=False)

In [5]:
# Count of records with missing date
report[report.Date == 'No Ship'].count()

Vendor       817
Receipts     817
Date         817
Shipments      0
dtype: int64

In [6]:
# Remove records without a date
report = report[report.Date != 'No Date']
report = report[report.Date != 'No Ship']

# Convert Dates
report['Date'] = pd.to_datetime(report['Date']).dt.date

In [7]:
# Add columns for year, quarter, month
report['Year'] = pd.to_datetime(report['Date']).dt.strftime('%Y')
report['Month'] = pd.to_datetime(report['Date']).dt.strftime('%m')
report['Quarter'] = pd.to_datetime(report['Date']).dt.quarter

# Different Reporting Options

In [8]:
# Group by that allows us to generate pivot tables and reports
grouped_report = report.groupby(['Vendor', 'Year', 'Quarter', 'Month']).agg({'Receipts':'sum', 'Shipments':'sum'})

# Compute GP
grouped_report['GP'] = grouped_report['Shipments'] - grouped_report['Receipts']

# Removes indices so we can use it in pivottablejs
final_report = grouped_report.reset_index()

In [9]:
# Pivottablejs
pivot_ui(final_report,
         cols=['Year', 'Quarter', 'Month'],
         rows=['Vendor'],
         vals=['GP'],
         aggregatorName='Sum',
         outfile_path='MFG Performance Final.html'
        )

In [10]:
# Another option for reporting
grouped_report

Receipts  Shipments  \
Vendor                 Year Quarter Month                            
Avnet Electronics      2019 1       01     3.435518e+05  100872.50   
                                    02     1.029341e+06   35720.00   
                                    03     1.185157e+05  531012.50   
                            2       04     0.000000e+00    6250.00   
                                    06     0.000000e+00    4350.00   
...                                                 ...        ...   
Skyhigh Memory Limited 2019 4       10     2.122032e+04   11040.00   
                                    11     3.720250e+03       0.00   
                       2020 1       02     1.026000e+04   14250.00   
                                    03     1.054774e+04   17723.75   
                            2       05     2.576700e+03    6863.50   

                                                      GP  
Vendor                 Year Quarter Month                 
Avnet Electronics      2019 1       01    -242679.333660  
                                    02    -993621.322827  
                                    03     412496.751200  
                            2       04       6250.000000  
                                    06       4350.000000  
...                                                  ...  
Skyhigh Memory Limited 2019 4       10     -10180.320000  
                                    11      -3720.250000  
                       2020 1       02       3990.000000  
                                    03       7176.010000  
                            2       05       4286.800000  

[139 rows x 3 columns]

In [11]:
# Generate a pivot table
table = pd.pivot_table(final_report, values=['Receipts', 'Shipments', 'GP'], index='Vendor',
                    columns=['Year', 'Quarter', 'Month'], aggfunc=np.sum, margins=True)

table

GP                            \
Year                                       2016      2017                   
Quarter                                       4         1       2           
Month                                        11        03      05      06   
Vendor                                                                      
Avnet Electronics                           NaN       NaN     NaN     NaN   
Cypress Semiconductor                       NaN       NaN     NaN     NaN   
GSI Technology                              NaN       NaN     NaN     NaN   
IDT                                     43200.0  322150.0  8100.0  2900.0   
Lattice Semiconductor Corp.                 NaN       NaN     NaN     NaN   
NXP                                         NaN       NaN     NaN     NaN   
Renesas Electronics America Inc.            NaN       NaN     NaN     NaN   
Renesas Electronics America, Inc (IDT)      NaN       NaN     NaN     NaN   
Skyhigh Memory Limited                      NaN       NaN     NaN     NaN   
All                                     43200.0  322150.0  8100.0  2900.0   

                                                                           \
Year                                                                        
Quarter                                       3                         4   
Month                                        07       08       09      10   
Vendor                                                                      
Avnet Electronics                           NaN      NaN      NaN     NaN   
Cypress Semiconductor                       NaN      NaN      NaN     NaN   
GSI Technology                              NaN      NaN      NaN     NaN   
IDT                                     9287.82  13600.0  34763.0  1615.0   
Lattice Semiconductor Corp.                 NaN      NaN      NaN     NaN   
NXP                                         NaN      NaN      NaN     NaN   
Renesas Electronics America Inc.            NaN      NaN      NaN     NaN   
Renesas Electronics America, Inc (IDT)      NaN      NaN      NaN     NaN   
Skyhigh Memory Limited                      NaN      NaN      NaN     NaN   
All                                     9287.82  13600.0  34763.0  1615.0   

                                                            ...   Shipments  \
Year                                                        ...        2019   
Quarter                                                     ...           3   
Month                                         11        12  ...          09   
Vendor                                                      ...               
Avnet Electronics                            NaN       NaN  ...    18879.00   
Cypress Semiconductor                   12513.26  14257.16  ...   477094.92   
GSI Technology                               NaN       NaN  ...    15642.50   
IDT                                     18598.72  29181.76  ...     5570.75   
Lattice Semiconductor Corp.                  NaN       NaN  ...         NaN   
NXP                                          NaN       NaN  ...   334740.15   
Renesas Electronics America Inc.             NaN       NaN  ...         NaN   
Renesas Electronics America, Inc (IDT)       NaN       NaN  ...   194048.48   
Skyhigh Memory Limited                       NaN       NaN  ...         NaN   
All                                     31111.98  43438.92  ...  1045975.80   

                                                                           \
Year                                                                        
Quarter                                          4                          
Month                                           10         11          12   
Vendor                                                                      
Avnet Electronics                          1301.75    3255.00         NaN   
Cypress Semiconductor                   1052917.07  386479.98   709551.90   
GSI Tech

In [12]:
# Group by the date only so we can send it to Excel for formatting
grouped_report = report.groupby(['Vendor', 'Date']).agg({'Receipts':'sum', 'Shipments':'sum'})
grouped_report['GP'] = grouped_report['Shipments'] - grouped_report['Receipts']

final_report = grouped_report.reset_index()

In [14]:
# Save to excel
final_report.to_excel('final_report_1.xlsx')
table.to_excel('final_report_2.xlsx')